In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import jax.numpy as jnp # this is a thin wrapper to NumPy within JAX
from jax import grad, hessian

mpl.rcParams['lines.linewidth'] = 2
mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18) 
mpl.rc('axes', labelsize=18) 
mpl.rc('font', size=18) 
plt.rcParams.update({
    'text.usetex': True,
    'font.family': 'serif',
})

# define objective function and constraints

In [2]:
f = lambda x: (3*x[...,0]**2 + x[...,1]**2 + 2*x[...,0]*x[...,1] + x[...,0] + 6*x[...,1]).squeeze()
c_1 = lambda x: (2.*x[...,0] + 3*x[...,1] - 4.).squeeze()
c_2 = lambda x: (x[...,0]).squeeze()
c_3 = lambda x: (x[...,1]).squeeze()
c = [c_1, c_2, c_3]
d = np.array([-4., 0., 0.]) # RHS of constraint equations

In [44]:
m = len(c)
n = 2
xk = np.array([[3., 2.]]) # starting point
Bk = hessian(f)(xk).squeeze() # function hessian
Ck = np.concatenate([grad(c[i])(xk) for i in range(m)]) # note (upper case) Ck has *gradients* of all constraints at xk
ck = np.concatenate([[c[i](xk)] for i in range(m)]) # note (lower case) ck has *value* of all constraints at xk

In [45]:
def get_active_set(ck, xk):
    Wk = []
    # find which constraints are active
    for i in range(m):
        if np.abs(ck[i]) <= 1e-12:
            Wk.append(i)
    return Wk

In [5]:
def construct_kkt_and_solve(xk, Bk, Ck, Wk, f, c):
    n_active = len(Wk)
    m    = len(c)
    lk_I = np.zeros(m) # Lagrange multipliers for all inequality constraints
    Z    = np.zeros([n_active, n_active])
    
    if n_active > 0:
        L = np.block([[Bk, -Ck[Wk].T],
                      [-Ck[Wk], Z]])
    else:
        L = Bk # no active constraints

    y    = np.append(-grad(f)(xk).squeeze() + np.matmul(Ck[Wk].T, lk_I[Wk]), np.zeros(n_active) )
    soln = np.linalg.solve(L, y)
    pk   = soln[:n]

    if n_active > 0:
        lk_I[Wk] = soln[-n_active:]
    
    return pk, lk_I

In [6]:
def update_lagrange_multipliers(lk_I, Wk):
    m    = lk_I.shape[0]
    mask = np.full(m, True, dtype=bool)
    mask[Wk]  = False # identify inactive constraints
    I_k       = np.arange(m)[mask]
    lk_I[I_k] = np.zeros_like(I_k)
    return lk_I

In [38]:
def get_blocking_constraints(Ck, pk):
    m = Ck.shape[0]
    cTp = []
    for i in range(m):
        cTp.append(np.dot(Ck[i], pk))
    blocking_ind = np.arange(m)[np.array(cTp) < -1e-12]
    # print(f'dot(c,p) {cTp} blocking constraints {blocking_ind}, {blocking_ind.shape}')
    return blocking_ind

In [35]:
def get_steplengths(blocking_ind, Ck, d, pk):
    m = Ck.shape[0]
    a = np.zeros(m)
    for i in blocking_ind:
        a[i] = -(np.dot(Ck[i],xk.squeeze()) + d[i]) / np.dot(Ck[i], pk) 
        # print(f'alpha_{i+1} = {a[i]}')
    ak = min(1, min(a[blocking_ind]))
    return ak

In [41]:
def remove_negative_lambda_constraints(lk_I, Wk):
    removed = False
    m = lk_I.shape[0]
    if any(lk_I < 0):
        print('negative Lagrange parameters, remove one of them from working set')
        negative_l_inds = np.arange(m)[lk_I < 0]
        if len(lk_I[negative_l_inds]) == 0:
            print('all Lagrange parameters are nonnegative')
        elif len(lk_I[negative_l_inds]) == 1:
            Wk.remove(negative_l_inds)
            removed = True
        else:
            Wk.remove(lk_I.argmin())
            removed = True
    else:
        print(f'Lagrange multipliers are all nonnegative!!')
    return Wk, removed

In [47]:
xk    = np.array([[3., 2.]]) # starting point
pkinf = 10
eps   = 1e-12
m     = len(c)
lk_I    = np.zeros(m)
removed = False
k     = 0

while pkinf > eps or any(lk_I < 0):

    Bk = hessian(f)(xk).squeeze() # function hessian
    Ck = np.concatenate([grad(c[i])(xk) for i in range(m)]) # gradients of each constraint
    ck = np.concatenate([[c[i](xk)] for i in range(m)]) # value of each constraint
    if ~removed:
        Wk          = get_active_set(ck, xk) 
    pk, lk_I    = construct_kkt_and_solve(xk, Bk, Ck, Wk, f, c)
    pkinf = np.linalg.norm(pk, np.inf)
    # print(f'Working set {Wk}, pk {pk}, Lagrange multipliers {lk_I}')
    
    if pkinf < eps:
        print(f'pk=0, potential convergence.')
        # check for Lagrange parameter sign
        Wk, removed = remove_negative_lambda_constraints(lk_I, Wk)
        pk, lk_I    = construct_kkt_and_solve(xk, Bk, Ck, Wk, f, c)
        lk_I        = update_lagrange_multipliers(lk_I, Wk)
        pkinf = np.linalg.norm(pk, np.inf)

    blocking_ind= get_blocking_constraints(Ck, pk)
    if blocking_ind.shape[0] > 0:
        ak = get_steplengths(blocking_ind, Ck, d, pk)
        # print(f'chosen step length alpha_k {ak}')
        xk = xk + ak * pk
        # print(f'pk {pk}, new point xk {xk}')
        removed = False
    k += 1
    
    print(f'iter {k}, Working set {Wk}, pk {pk}, step length {ak}, new point {xk}, Lagrange multipliers {lk_I}')

print('-------------------------------')
print(f'constrained minimizer {xk}, constrained minimum {f(xk)}, Lagrange multipliers {lk_I}')

iter 1, Working set [], pk [-1.75 -6.25], step length 0.32, new point [[2.44 0.  ]], Lagrange multipliers [0. 0. 0.]
iter 2, Working set [2], pk [-2.60666672e+00  6.66133815e-16], step length 0.16879795025881222, new point [[2.00000000e+00 1.12442023e-16]], Lagrange multipliers [0.         0.         5.66666667]
pk=0, potential convergence.
negative Lagrange parameters, remove one of them from working set
iter 3, Working set [0], pk [-1.5  1. ], step length 1, new point [[0.5 1. ]], Lagrange multipliers [3. 0. 0.]
pk=0, potential convergence.
Lagrange multipliers are all nonnegative!!
iter 4, Working set [0], pk [-2.06184276e-16  1.74463618e-16], step length 1, new point [[0.5 1. ]], Lagrange multipliers [3. 0. 0.]
-------------------------------
constrained minimizer [[0.5 1. ]], constrained minimum 9.249999999999998, Lagrange multipliers [3. 0. 0.]
